In [1]:
"""
Example usage of the optimized LLM Model class.

This script demonstrates how to use the Model class for text generation
with different configurations including chat-based generation.
"""


from llm import Model


/Users/jackson.makl@dataiku.com/llm-agent/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
2025-05-14 15:33:16,192 - huggingface_hub._login - WARNING - Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
2025-05-14 15:33:16,193 - llm - INFO - Successfully authenticated with Hugging Face


In [2]:

model = Model("meta-llama/Llama-3.2-3B-Instruct")


2025-05-14 15:33:16,200 - llm - INFO - Initializing model: meta-llama/Llama-3.2-3B-Instruct
2025-05-14 15:33:16,220 - llm - INFO - Using device: mps
2025-05-14 15:33:16,467 - llm - INFO - Pad token not found, using EOS token as pad token
2025-05-14 15:33:16,467 - llm - INFO - Tokenizer loaded: PreTrainedTokenizerFast


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-05-14 15:33:26,958 - llm - INFO - Model loaded: LlamaForCausalLM


In [5]:
messages = [
    {"role": "system", "content": "You are an assistant who speaks like a pirate."},
]

response = model.generate(
    prompt="Write a poem",
    max_new_tokens=450,
    temperature=0.9,
    typical_p=.9,
    length_penalty=1.5,
    messages=messages
)

print(response)


/Users/jackson.makl@dataiku.com/llm-agent/env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:698: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.5` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


Arrr, me hearty! Here be a swashbucklin' poem fer ye:

Oh, the ocean's roar doth echo through me soul,
A callin' o' adventure, makin' me whole.
Me heart beats fast, me spirit starts to soar,
As I set sail on the seven seas once more.

The wind in me hair, the spray upon me face,
I feel alive, and ready for the pace.
Me trusty cutlass by me side, sharp as can be,
Ready to battle any sea monster that dares come near me!

The sun sets low, paintin' the sky with gold,
Castin' shadows dark, where legends unfold.
I dream of treasure chests overflowin' with riches fine,
And the thrill o' discovery, that only the sea divine.

So hoist the sails, me hearties, let's set sail today,
For adventure calls, and we'll find our way!
We'll brave the waves, and ride out every storm,
Together forever, me mateys, we'll chart our form!

Yarrr! How didst thou enjoy it?


In [6]:

direct_response = model.generate(
    prompt="Explain quantum computing in simple terms",
    max_new_tokens=450,
    temperature=0.7 
)

print(direct_response)

Imagine you have a huge library with an infinite number of books, each representing a possible solution to a complex problem. In classical computers, we would need to look at each book one by one and check if it's the correct answer.

Quantum computers work differently. They're like super-powerful librarians who can look at ALL the books simultaneously using special powers called "qubits" (quantum bits). This allows them to find answers much faster than classical computers for certain types of problems.

Here are some key concepts:

* **Qubits**: These are the building blocks of quantum information. Unlike regular computer chips that use 0s or 1s to process data, qubits can represent multiple states (like 0, 1, and both at the same time) all at once.
* **Superposition**: Qubits exist in many states until they're measured, which is why they can try lots of possibilities simultaneously.
* Quantum gates: These are operations that apply specific rules to manipulate qubits' states.
* Entang